# <center> Desafio Técnico - Cientista de Dados - Americanas S.A.

<p style='text-align: justify;'> O objetivo deste trabalho é apresentar a minha solução para o Desafio Técnico do processo seletivo para a posição de Cientista de Dados Júnior na Americanas S.A.

Este documento está dividido em quatro partes : **Análise Exploratória dos Dados** , **Preparação dos Dados** , **Modelagem** e **Avaliação da Performance do Modelo**.

Em um notebook separado está localizada a parte final do desafio, ou seja, a **Entrega do Modelo**. Esse notebook pode ser encontrado no repositório criado para hospedar minha solução cujo endereço é https://github.com/LucasPorto2/Desafio-Tecnico-Americanas-S.A.

In [1]:
# Importando algumas bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier , GradientBoostingClassifier , VotingClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score , GridSearchCV

## <center>  1. Análise Exploratória dos Dados

### 1.1 Leitura do Arquivo

In [2]:
# Lendo o conjunto de dados e transformando o mesmo em um dataframe

df = pd.read_parquet('dataset_cdjr.parquet.gzip' , engine = 'pyarrow')

### 1.2 Observações Iniciais

In [3]:
# Verificando a dimensão do dataset

df.shape

(466, 17)

De fato existem 466 entradas e 17 colunas (16 features e uma variável alvo) conforme informado !

In [4]:
# Observando as 5 primeiras linhas do nosso dataset

df.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,target
337,200.0,2,662.28,39.10,-188.55,0.246978,761,0.004548,3.523703,167326,33441.06,0.019804,26.850,0.009198,94.611429,7,0
266,150.0,2,0.00,149.55,-0.45,0.150000,3,0.037975,0.000000,79,78.93,0.000000,0.000,0.000000,0.000000,0,1
236,50.0,1,346.08,30.41,-102.10,2.430952,42,0.004239,3.389618,9907,18858.77,0.018351,25.525,0.095238,86.520000,4,0
274,100.0,2,0.00,43.84,-56.16,0.150968,372,0.005854,0.000000,63544,1164.11,0.000000,0.000,0.000000,0.000000,0,1
208,50.0,1,87.56,-3.05,-94.50,0.412664,229,0.004572,0.926561,50089,1786.26,0.049019,94.500,0.004367,87.560000,1,0


In [5]:
# Verificando o número de valores ausentes por coluna 

df.isnull().sum()

feature0     0
feature1     0
feature2     0
feature3     0
feature4     0
feature5     0
feature6     0
feature7     0
feature8     0
feature9     0
feature10    0
feature11    0
feature12    0
feature13    0
feature14    0
feature15    0
target       0
dtype: int64

Que ótimo ! Não possuimos valores faltantes em nosso conjunto de dados logo não precisamos nos preocupar em lidar com valores ausentes.

In [6]:
# Obtendo algumas estatísticas básicas 

df.describe()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,target
count,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,4.660000e+02,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000
mean,438.743562,4.847639,1244.322468,44.600880,-434.299893,0.473991,979.070815,0.113885,3.857010,2.164378e+05,6930.456438,0.437333,61.886190,0.008634,87.713360,10.313305,0.557940
std,984.593065,6.836679,3558.699033,122.093515,975.555198,0.452261,1460.738442,1.873746,15.609132,3.508622e+05,17581.800818,3.442094,142.521523,0.017866,145.426437,33.625204,0.497165
min,0.000000,0.000000,0.000000,-645.870000,-15506.350000,0.150000,1.000000,0.000663,0.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,1.000000,0.000000,-0.400000,-492.035000,0.173669,133.500000,0.003318,0.000000,2.113100e+04,383.687500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,150.000000,2.000000,169.830000,18.360000,-154.525000,0.303854,420.000000,0.005127,0.937082,8.532850e+04,1410.855000,0.082515,19.692568,0.003207,72.524286,2.000000,1.000000
75%,500.000000,6.000000,1017.375000,44.630000,-50.180000,0.569848,1238.750000,0.009699,2.815824,2.645035e+05,5212.977500,0.311311,65.498098,0.009515,127.301505,8.000000,1.000000
max,15400.000000,31.000000,40291.240000,1521.900000,-0.260000,3.150000,11731.000000,40.000000,281.666667,3.366472e+06,237182.780000,73.080634,2232.100000,0.204611,2154.000000,541.000000,1.000000


<p style='text-align: justify;'>Com apenas algumas estatísticas básicas percebemos que a maioria das features possuí alta variância. Por exemplo a feature9 assume valores entre 1 e 3366472. Isso já é um forte indicativo que devemos nos preocupar com o dimensionamento dos dados, porém isso será melhor abordado na seção Feature Scaling dentro do tópico Preparação dos Dados.

In [7]:
# Observando os tipos de dado em nosso dataset

df.dtypes

feature0     float64
feature1       int64
feature2     float64
feature3     float64
feature4     float64
feature5     float64
feature6       int64
feature7     float64
feature8     float64
feature9       int64
feature10    float64
feature11    float64
feature12    float64
feature13    float64
feature14    float64
feature15      int64
target         int64
dtype: object

In [8]:
# Verificando como de fato todas as entradas da variável target são 0s ou 1s

df['target'].value_counts()

1    260
0    206
Name: target, dtype: int64

In [9]:
# % de exemplos onde target = 1

(260/466)*100

55.793991416309005

In [10]:
# % de exemplos onde target = 0

(206/466)*100

44.20600858369099

## <center> 2. Preparação dos Dados

### 2.1 Separando o dataset em Features e Target

In [11]:
# Criando a matriz de features

X = np.array(df.drop('target' , axis = 1))

In [12]:
# Criando o array target

target = np.array(df['target'])

### 2.2 Feature Scaling

<p style='text-align: justify;'> O dimensionamento dos dados ou Feature Scaling é algo sempre muito importante para levarmos em conta em um projeto de Ciência de Dados. Quando certas variáveis assumem valores em um intervalo muito grande isso pode acabar impactando o treinamento dos modelos uma vez que muitos algoritmos de Machine Learning calculam distâncias entre pontos das amostras e se os dados estiverem em escalas muito diferentes podemos ter distâncias muito elevadas o que acaba por gerar problemas em nossas previsões.

<p style='text-align: justify;'> Para realizar essa etapa vamos utilizar o StandardScaler do Scikit-Learn para obrigar nossas variáveis a terem média 0 e desvio-padrão 1 (aproximar uma distribuição normal padrão).

In [13]:
# Instanciando o método

SS = StandardScaler()

In [14]:
# Feature Scaling

X = SS.fit_transform(X)

In [15]:
# Matriz transformada

X

array([[-0.24274002, -0.41697144, -0.16373062, ...,  0.03161502,
         0.04748436, -0.09864224],
       [-0.29357699, -0.41697144, -0.35003233, ..., -0.48378748,
        -0.6037941 , -0.30704315],
       [-0.39525095, -0.56339849, -0.2526788 , ...,  4.85255679,
        -0.00821475, -0.18795692],
       ...,
       [-0.03939211, -0.12411735, -0.04057873, ...,  0.18553215,
         0.47800197, -0.09864224],
       [-0.29357699, -0.56339849, -0.35003233, ..., -0.48378748,
        -0.6037941 , -0.30704315],
       [-0.39525095, -0.56339849, -0.35003233, ..., -0.48378748,
        -0.6037941 , -0.30704315]])

In [16]:
# Testando se de fato a primeira coluna possui média 0 e desvio-padrão 1

np.mean(X[: , 0]) , np.std(X[: , 0])

(0.0, 1.0)

Tudo correto !

## <center> 3. Modelagem

In [17]:
# Modelos a serem testados

modelos = [LogisticRegression() , KNeighborsClassifier() , SVC() , DecisionTreeClassifier() ,
           RandomForestClassifier() , AdaBoostClassifier() , GradientBoostingClassifier() ,
           XGBClassifier(use_label_encoder = False) , LGBMClassifier(), CatBoostClassifier()]

Vamos inicialmente medir a acurácia de cada modelo no conjunto de treinamento, ou seja, vamos treinar o modelo no dataset inteiro (466 amostras) e vamos medir a acurácia que o modelo obtém quando fazemos previsões no próprio conjunto de treino. Para isso criaremos uma lista acc_treino inicialmente vazia.

In [18]:
acc_treino = []

In [19]:
# Treinando cada modelo, medindo sua acurácio e guardando na lista acc_treino

for model in modelos :
    model.fit(X, target)
    acc_treino.append(model.score(X, target))

[03:48:00] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.007436
0:	learn: 0.6919471	total: 197ms	remaining: 3m 16s
1:	learn: 0.6905011	total: 210ms	remaining: 1m 44s
2:	learn: 0.6893491	total: 226ms	remaining: 1m 15s
3:	learn: 0.6879204	total: 240ms	remaining: 59.7s
4:	learn: 0.6865646	total: 254ms	remaining: 50.5s
5:	learn: 0.6848357	total: 268ms	remaining: 44.4s
6:	learn: 0.6829308	total: 282ms	remaining: 40s
7:	learn: 0.6821643	total: 295ms	remaining: 36.6s
8:	learn: 0.6806840	total: 309ms	remaining: 34s
9:	learn: 0.6792351	total: 324ms	remaining: 32s
10:	learn: 0.6780155	total: 338ms	remaining: 30.4s
11:	learn: 0.6767936	total: 351ms	remaining: 28.9s
12:	learn: 0.6754268	total: 365ms	remaining: 27.7s
13:	learn: 0.6738677	total: 379ms	remaining: 26.7s
14:	learn: 0.6727

159:	learn: 0.5483776	total: 2.63s	remaining: 13.8s
160:	learn: 0.5476587	total: 2.64s	remaining: 13.8s
161:	learn: 0.5469019	total: 2.66s	remaining: 13.8s
162:	learn: 0.5464025	total: 2.68s	remaining: 13.8s
163:	learn: 0.5459364	total: 2.69s	remaining: 13.7s
164:	learn: 0.5454833	total: 2.71s	remaining: 13.7s
165:	learn: 0.5450015	total: 2.72s	remaining: 13.7s
166:	learn: 0.5443010	total: 2.74s	remaining: 13.7s
167:	learn: 0.5440958	total: 2.75s	remaining: 13.6s
168:	learn: 0.5437624	total: 2.77s	remaining: 13.6s
169:	learn: 0.5433421	total: 2.78s	remaining: 13.6s
170:	learn: 0.5428974	total: 2.79s	remaining: 13.5s
171:	learn: 0.5424637	total: 2.81s	remaining: 13.5s
172:	learn: 0.5419747	total: 2.83s	remaining: 13.5s
173:	learn: 0.5412840	total: 2.85s	remaining: 13.5s
174:	learn: 0.5408359	total: 2.86s	remaining: 13.5s
175:	learn: 0.5402849	total: 2.87s	remaining: 13.5s
176:	learn: 0.5398002	total: 2.89s	remaining: 13.4s
177:	learn: 0.5391779	total: 2.9s	remaining: 13.4s
178:	learn: 0

321:	learn: 0.4747175	total: 5.04s	remaining: 10.6s
322:	learn: 0.4743182	total: 5.06s	remaining: 10.6s
323:	learn: 0.4739985	total: 5.08s	remaining: 10.6s
324:	learn: 0.4736159	total: 5.09s	remaining: 10.6s
325:	learn: 0.4733088	total: 5.11s	remaining: 10.6s
326:	learn: 0.4728614	total: 5.12s	remaining: 10.5s
327:	learn: 0.4724480	total: 5.14s	remaining: 10.5s
328:	learn: 0.4722143	total: 5.15s	remaining: 10.5s
329:	learn: 0.4717670	total: 5.16s	remaining: 10.5s
330:	learn: 0.4713715	total: 5.18s	remaining: 10.5s
331:	learn: 0.4708150	total: 5.19s	remaining: 10.4s
332:	learn: 0.4704227	total: 5.21s	remaining: 10.4s
333:	learn: 0.4700822	total: 5.22s	remaining: 10.4s
334:	learn: 0.4698303	total: 5.24s	remaining: 10.4s
335:	learn: 0.4694554	total: 5.26s	remaining: 10.4s
336:	learn: 0.4691409	total: 5.27s	remaining: 10.4s
337:	learn: 0.4687304	total: 5.29s	remaining: 10.4s
338:	learn: 0.4685656	total: 5.3s	remaining: 10.3s
339:	learn: 0.4682860	total: 5.31s	remaining: 10.3s
340:	learn: 0

481:	learn: 0.4214917	total: 7.44s	remaining: 7.99s
482:	learn: 0.4212768	total: 7.45s	remaining: 7.98s
483:	learn: 0.4210115	total: 7.47s	remaining: 7.96s
484:	learn: 0.4203825	total: 7.49s	remaining: 7.95s
485:	learn: 0.4201281	total: 7.5s	remaining: 7.93s
486:	learn: 0.4197244	total: 7.52s	remaining: 7.92s
487:	learn: 0.4194956	total: 7.53s	remaining: 7.9s
488:	learn: 0.4191567	total: 7.54s	remaining: 7.88s
489:	learn: 0.4188524	total: 7.56s	remaining: 7.87s
490:	learn: 0.4185775	total: 7.57s	remaining: 7.85s
491:	learn: 0.4181282	total: 7.59s	remaining: 7.83s
492:	learn: 0.4178314	total: 7.6s	remaining: 7.82s
493:	learn: 0.4173784	total: 7.61s	remaining: 7.8s
494:	learn: 0.4170097	total: 7.63s	remaining: 7.79s
495:	learn: 0.4166377	total: 7.65s	remaining: 7.77s
496:	learn: 0.4162923	total: 7.67s	remaining: 7.76s
497:	learn: 0.4160730	total: 7.68s	remaining: 7.74s
498:	learn: 0.4156808	total: 7.69s	remaining: 7.72s
499:	learn: 0.4153898	total: 7.71s	remaining: 7.71s
500:	learn: 0.41

652:	learn: 0.3724276	total: 10s	remaining: 5.32s
653:	learn: 0.3722125	total: 10s	remaining: 5.3s
654:	learn: 0.3720200	total: 10s	remaining: 5.29s
655:	learn: 0.3717407	total: 10.1s	remaining: 5.28s
656:	learn: 0.3713962	total: 10.1s	remaining: 5.26s
657:	learn: 0.3709826	total: 10.1s	remaining: 5.24s
658:	learn: 0.3707480	total: 10.1s	remaining: 5.23s
659:	learn: 0.3705335	total: 10.1s	remaining: 5.21s
660:	learn: 0.3701409	total: 10.1s	remaining: 5.2s
661:	learn: 0.3698143	total: 10.1s	remaining: 5.18s
662:	learn: 0.3695166	total: 10.2s	remaining: 5.16s
663:	learn: 0.3693917	total: 10.2s	remaining: 5.15s
664:	learn: 0.3692026	total: 10.2s	remaining: 5.13s
665:	learn: 0.3687454	total: 10.2s	remaining: 5.12s
666:	learn: 0.3685588	total: 10.2s	remaining: 5.1s
667:	learn: 0.3683430	total: 10.2s	remaining: 5.09s
668:	learn: 0.3681010	total: 10.3s	remaining: 5.07s
669:	learn: 0.3677982	total: 10.3s	remaining: 5.06s
670:	learn: 0.3674908	total: 10.3s	remaining: 5.04s
671:	learn: 0.3670728

814:	learn: 0.3324775	total: 12.4s	remaining: 2.82s
815:	learn: 0.3321820	total: 12.5s	remaining: 2.81s
816:	learn: 0.3319058	total: 12.5s	remaining: 2.79s
817:	learn: 0.3317201	total: 12.5s	remaining: 2.78s
818:	learn: 0.3314862	total: 12.5s	remaining: 2.76s
819:	learn: 0.3312648	total: 12.5s	remaining: 2.75s
820:	learn: 0.3310345	total: 12.5s	remaining: 2.73s
821:	learn: 0.3307695	total: 12.5s	remaining: 2.71s
822:	learn: 0.3305784	total: 12.6s	remaining: 2.7s
823:	learn: 0.3302843	total: 12.6s	remaining: 2.68s
824:	learn: 0.3300227	total: 12.6s	remaining: 2.67s
825:	learn: 0.3299083	total: 12.6s	remaining: 2.65s
826:	learn: 0.3297421	total: 12.6s	remaining: 2.64s
827:	learn: 0.3296292	total: 12.6s	remaining: 2.62s
828:	learn: 0.3294711	total: 12.6s	remaining: 2.61s
829:	learn: 0.3292618	total: 12.7s	remaining: 2.59s
830:	learn: 0.3289442	total: 12.7s	remaining: 2.58s
831:	learn: 0.3285942	total: 12.7s	remaining: 2.56s
832:	learn: 0.3283506	total: 12.7s	remaining: 2.55s
833:	learn: 0

974:	learn: 0.2940149	total: 14.8s	remaining: 380ms
975:	learn: 0.2937616	total: 14.8s	remaining: 365ms
976:	learn: 0.2936273	total: 14.8s	remaining: 349ms
977:	learn: 0.2933891	total: 14.9s	remaining: 335ms
978:	learn: 0.2932187	total: 14.9s	remaining: 320ms
979:	learn: 0.2930476	total: 15s	remaining: 305ms
980:	learn: 0.2927750	total: 15s	remaining: 290ms
981:	learn: 0.2924937	total: 15s	remaining: 275ms
982:	learn: 0.2922248	total: 15s	remaining: 259ms
983:	learn: 0.2920201	total: 15s	remaining: 244ms
984:	learn: 0.2917911	total: 15s	remaining: 229ms
985:	learn: 0.2914666	total: 15.1s	remaining: 214ms
986:	learn: 0.2913760	total: 15.1s	remaining: 198ms
987:	learn: 0.2910505	total: 15.1s	remaining: 183ms
988:	learn: 0.2907665	total: 15.1s	remaining: 168ms
989:	learn: 0.2905886	total: 15.1s	remaining: 153ms
990:	learn: 0.2903488	total: 15.1s	remaining: 137ms
991:	learn: 0.2899092	total: 15.1s	remaining: 122ms
992:	learn: 0.2896304	total: 15.1s	remaining: 107ms
993:	learn: 0.2894392	to

In [20]:
# Criando uma lista contendo o nome de cada modelo

nome_modelo = ['LogisticRegression' , 'KNeighborsClassifier' , 'SVC' , 'DecisionTreeClassifier' ,
           'RandomForestClassifier' , 'AdaBoostClassifier' , 'GradientBoostingClassifier' ,
           'XGBClassifier' , 'LGBMClassifier', 'CatBoostClassifier']

In [21]:
# Criando um dataframe para visualizar a acurácia de cada modelo ao fazermos previsões no conjunto de treino

pd.DataFrame({'Modelo' : nome_modelo , 'Acurácia no conjunto de treino' : acc_treino})

,Modelo,Acurácia no conjunto de treino
0,LogisticRegression,0.637339
1,KNeighborsClassifier,0.736052
2,SVC,0.675966
3,DecisionTreeClassifier,1.000000
4,RandomForestClassifier,1.000000
5,AdaBoostClassifier,0.804721
6,GradientBoostingClassifier,0.937768
7,XGBClassifier,1.000000
8,LGBMClassifier,1.000000
9,CatBoostClassifier,0.954936


<p style='text-align: justify;'> Podemos observar que treinamos 10 modelos distintos e 4 deles nos deram acurácia de 100% , ou seja, 4 modelos acertaram os 466 exemplos e nem precisamos otimizar hiperparâmetros. Isso é excelente ! </p>
    

<p style='text-align: justify;'>Devemos nos lembrar que a acurácia obtida quando fazemos previsões no conjunto de treino é praticamente sempre maior que a obtida em um conjunto de dados não vistos, pois um determinado modelo pode sofrer de overfitting, isto é, pode estar apenas "indo bem" nos dados vistos e pode não conseguir generalizar tão bem para dados não vistos pelo modelo. Isso é comum e seria como se o modelo estivesse apenas decorando os exemplos do conjunto de treino, o que pode acontecer uma vez que o dataset é pequeno. Será que os nossos modelos generalizam bem para dados não vistos ? É o que veremos na próxima seção !

<p style='text-align: justify;'>Antes preciso ressaltar que se o objetivo for apenas medir a acurácia no conjunto de treino então qualquer um dos 4 modelos que obtém acurácia de 100% (Árvore de decisão , Random Forest, XGB e LGBM) é uma boa escolha ! Na próxima seção verificaremos se o Random Forest, XGB e LGBM generalizam bem para dados não vistos e deixaremos de fora o modelo de Árvore de Decisão, pois esse tipo de modelo costuma sofrer de overfitting (árvores muito profundas decoram os exemplos de treino, mas possuem dificuldade de generalizar).

## <center> 4. Avaliação da Performance do Modelo

<p style='text-align: justify;'>Já avaliamos a performance dos nossos modelos quando treinamos no dataset inteiro. Agora utilizaremos uma técnica chamada validação cruzada que permite treinarmos nossos modelos em diferentes partes do dataset e avaliarmos a performance na parte restante do dataset (parte que ficou de fora do treinamento). Vamos inicialmente utilizar uma validação cruzada com 5 dobras. Essa técnica nos dará uma noção melhor de como o modelo está generalizando, pois faremos diferentes treinamentos (um em cada parte do dataset) e também avaliaremos a performance em distintas dobras.

### Random Forest

In [26]:
# Acurácia do Random Forest nas diferentes dobras

RandomForest_acc = cross_val_score(RandomForestClassifier() , X , target , cv = 5)

RandomForest_acc

array([0.63829787, 0.66666667, 0.69892473, 0.59139785, 0.68817204])

In [27]:
# Acurácia média das 5 dobras

np.mean(RandomForest_acc)

0.6566918325326012

### XGBClassifier

In [28]:
# Acurácia do XGBClassifier nas diferentes dobras

XGB_acc = cross_val_score(XGBClassifier(use_label_encoder = False) , X , target , cv = 5)

XGB_acc

[03:50:30] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:50:31] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:50:31] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:50:32] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

array([0.68085106, 0.64516129, 0.67741935, 0.66666667, 0.66666667])

In [29]:
# Acurácia média das 5 dobras

np.mean(XGB_acc)

0.6673530084648821

### LightGBM

In [30]:
# Acurácia do LGBMClassifier nas diferentes dobras

LGBM_acc = cross_val_score(LGBMClassifier() , X , target , cv = 5)

LGBM_acc

array([0.65957447, 0.60215054, 0.6344086 , 0.62365591, 0.72043011])

In [31]:
# Acurácia média das 5 dobras

np.mean(LGBM_acc)

0.6480439258750857

### Considerações

<p style='text-align: justify;'> Como podemos perceber a performance dos 3 modelos é bem diferente quando avaliamos em dados vistos durante o treinamento e quando avaliamos em dados não vistos. Os 3 modelos acertam todos os exemplos vistos durante o treinamento e acertam em média aproximadamente 2 de cada 3 exemplos não vistos. Bem, acertar 2 de cada 3 exemplos não vistos durante o treinamento não é um resultado ruim. Isso acontece porque o dataset é bem pequeno, seriam necessárias mais que 466 amostras para construirmos um modelo mais potente e capaz de generalizar melhor.

<p style='text-align: justify;'> Dos 3 modelos o que gerou a acurácia mais alta foi o XGBClassifier com aproximadamente 66,74%. Vamos tentar aumentar um pouco mais a acurácia otimizando os hiperparâmetros desse modelo :

### Otimização de Hiperparâmetros com GridSearchCV

In [32]:
# Criando uma primeira grade para tentarmos otimizar o número de estimadores e a taxa de aprendizagem

param_grid1 = {'n_estimators' : [100, 200, 300, 400, 500, 600, 700, 800] ,
             'learning_rate' : [0.1 , 0.2 , 0.3 , 0.4 ,0.5 , 0.6]}

In [33]:
gs1 = GridSearchCV(XGBClassifier(use_label_encoder = False), param_grid = param_grid1 , cv = 5)

In [34]:
# Pode levar alguns minutos...

gs1.fit(X, target)

[03:55:09] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:55:10] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:55:10] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:55:11] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[03:55:48] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:55:50] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:55:51] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:55:53] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[03:56:22] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:56:24] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:56:25] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:56:26] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[03:56:55] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:56:56] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:56:57] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:56:58] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[03:57:26] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:57:27] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:57:27] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:57:28] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[03:57:57] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:57:58] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:57:58] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:57:59] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[03:58:26] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:58:28] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:58:29] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:58:29] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[03:58:51] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:58:52] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:58:53] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:58:54] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     use_label_encoder=False,
                                     validate_parameters=None, verbosity=None),
             param_grid={'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
                         'n_estimators': [100, 200, 300, 400, 500, 600, 700,
                                          800]})

In [35]:
# Observando o melhor score obtido

gs1.best_score_

0.6802791123312742

In [36]:
# Descobrindo quais hiperparâmetros nos permitem obter o melhor score

gs1.best_params_

{'learning_rate': 0.5, 'n_estimators': 500}

Repare que conseguimos obter uma acurácia média de 0.6802 que é superior a 0.6673 obtido com o XGB na sua configuração padrão.

Será que conseguimos melhorar ainda mais a acurácia modificando o hiperparâmetro max_depth ?

In [37]:
# Criando uma segunda grade para tentarmos otimizar o hiperparâmetro max_depth

param_grid2 = {'max_depth' : range(1, 11)}

In [38]:
gs2 = GridSearchCV(XGBClassifier(use_label_encoder = False, learning_rate = 0.5 , n_estimators = 500),
                   param_grid = param_grid2 , cv = 5)

In [39]:
gs2.fit(X, target)

[04:00:02] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:02] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:03] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:04] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[04:00:30] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:31] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:32] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:33] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.5, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=500, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     use_label_encoder=False,
                                     validate_parameters=None, verbosity=None),
             param_grid={'max_depth': range(1, 11)})

In [40]:
# Observando o melhor score obtido

gs2.best_score_

0.6802791123312742

In [41]:
# Descobrindo qual hiperparâmetro nos permitem obter o melhor score

gs2.best_params_

{'max_depth': 6}

<p style='text-align: justify;'> O valor de max_depth na configuração padrão do XGBClassifier, ou seja, max_depth = 6 é quem gera o melhor score quando learning_rate = 0.5 e n_estimators = 500

Podemos concluir então que uma boa escolha de modelo é um XGBClassifier com os seguintes hiperparâmetros :

**n_estimators = 500**

**learning_rate = 0.5**

**max_depth = 6**